In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import h5py

In [2]:
cd /home/chenxupeng/projects/deepshape/

/home/chenxupeng/projects/deepshape


In [3]:
def append_extra_line(f):
    """Yield an empty line after the last line in the file
    """
    for line in f:
        yield line
    yield ''
def read_fasta(filename):
    with open(filename, 'r') as f:
        name = None
        seq = ''
        for line in append_extra_line(f):
            if line.startswith('>') or (len(line) == 0):
                if (len(seq) > 0) and (name is not None):
                    yield (name, seq)
                if line.startswith('>'):
                    name = line.strip()[1:].split()[0]
                    seq = ''
            else:
                if name is None:
                    raise ValueError('the first line does not start with ">"')
                seq += line.strip()

In [4]:
#sequence file
fasta = dict(read_fasta('data/fasta/CDS.transcript.fa'))
#icshape file
f =h5py.File('/home/chenxupeng/projects/deepshape/data/icSHAPE/Spitale_2015_invitro/CDS')

In [5]:
#we should pick according to icshape's rna name
icshape = f['feature/icshape'][:]
name = f['name'][:]
start = f['start'][:]
end = f['end'][:]
rna_length = end-start
icshape.shape

(12676378,)

In [6]:
icshape

array([ 0.063,  0.051,  0.   , ...,  1.   ,  0.177,  0.   ], dtype=float32)

根据percentile转化icshape的不满足要求的位置为nan  percentile定5或者10
但是首先两类不均衡，要把0随机扔掉，直到和1的比例一样

In [7]:
print '1的个数: %d' %np.sum(np.where(icshape==1)[0].shape[0])
print '0的个数: %d' %np.sum(np.where(icshape==0)[0].shape[0])
print '随机扔掉的个数: %d' %(np.sum(np.where(icshape==0)[0].shape[0]) - np.sum(np.where(icshape==1)[0].shape[0]))
index_zero = np.where(icshape==0)[0]
index_zero.shape
np.random.seed(1234)
indd = np.arange(4271102)
np.random.shuffle(indd)
pick_zero_to_nan = index_zero[indd]
icshape[pick_zero_to_nan] = np.nan

1的个数: 568751
0的个数: 4839853
随机扔掉的个数: 4271102


In [8]:
def shape_tonan(percentile):
    low = np.nanpercentile(icshape,percentile)
    high = np.nanpercentile(icshape,100-percentile)
    index_nan = np.where(np.logical_and(icshape <high, icshape >low)==1)
    icshape_ = np.copy(icshape)
    icshape_[index_nan] = np.nan
    return icshape_

In [9]:
np.nanpercentile(icshape,90)

0.83799999952316284

In [10]:
#先取icshape，按照rna的name分别取出128的score加上name和起始位置，并且按照rna返回最终选的片段的起始位点，供选取sequence以及计算acc使用
#以cds的rna 10123条为顺序，一条一条rna来：
#每条rna切成128的小段，one hot，这是 128*4
#cds 文件怎么找icshape呢？  找到name的position i ,然后start[i] 是起始，end[i]是终止
#把这段icshape取出来， length， length -128 +1 为需要截取的次数 count
#for i in range(count): 取出y  把rna name 和在rna中的起始位置（i）加到前两维，算index: 用coverage给，如果非nan比例高于coverage的
#才予以保留   找到该条rna需要保留的i 
#然后构造X  X通过i找到起始位点，每次截取128的长度， 128*4  索引为rna name加上从第几个截取
#至此一个rna的处理完毕
nan_icshape = shape_tonan(5)  #其实全是0 所以nan的比例比90%少
coverage = 0.5

rna_icshape_whole = {}
rna_need_index = {}
rna_nan_index = {}
rna_index = np.array([])
for i in tqdm(range(name.shape[0])):   #name.shape[0]
    rna_name = name[i]
    sequence = fasta[rna_name]
    shape_score = nan_icshape[start[i]:end[i]]
    length = end[i] - start[i] 
    count = length-128+1
    if count >=0:
        y = np.zeros([count,130]).astype('S')
        index = np.array([])
        for j in range(count):
            y[j][0] = rna_name
            y[j][1] = j
            y[j][2:] = shape_score[j:j+128]  
            if np.where(y[j][2:] == str(np.nan))[0].shape[0]/128.0 >1 - coverage:
                index = np.concatenate((index,np.array([j])))
        y_ = np.delete(y,index,axis =0)
        index_ = np.setdiff1d(np.arange(count),index)
        rna_icshape_whole[i] = y_
        rna_need_index[i] = index_
        rna_nan_index[i] = index
        #符合大于128的rna的index
        rna_index = np.concatenate((rna_index,np.array([i])))
rna_index = rna_index.astype('int')

/home/chenxupeng/apps/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in less
  after removing the cwd from sys.path.
/home/chenxupeng/apps/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in greater
  after removing the cwd from sys.path.
  2%|▏         | 198/10123 [00:10<08:54, 18.56it/s]

KeyboardInterrupt: 

  2%|▏         | 198/10123 [00:30<25:05,  6.59it/s]

In [ ]:
sum_1 = 0
for i in range(rna_index.shape[0]):
    sum_1 +=rna_need_index[rna_index[i]].shape[0]
sum_2 = 0
for i in range(rna_index.shape[0]):
    sum_2 +=rna_nan_index[rna_index[i]].shape[0]

In [33]:
float(sum_1)/float(sum_1+sum_2)

0.03581034535713815

In [ ]:
print np.sum(np.isnan(icshape))/float(icshape.shape[0])
print np.sum(np.isnan(nan_icshape))/float(nan_icshape.shape[0])

In [37]:
class Onehot(object):
    def __init__(self, alphabet='ATCG'):
        self.transtable = np.zeros(128, np.int64)
        self.transtable[[ord(a) for a in alphabet]] = range(len(alphabet))
        self.alphabet = np.frombuffer(alphabet, np.int8)
    def encode(self, seq):
        seq_int = self.transtable[np.frombuffer(seq, np.int8)]
        encoded = np.zeros((len(seq), len(self.alphabet)), np.int8)
        encoded[np.r_[:len(seq)], seq_int] = 1
        return encoded
    def decode(self, a):
        return self.alphabet[np.argmax(a, axis=1)].tostring()

In [40]:
rna_index.shape[0]

9957

In [41]:
#i 使用符合rna长度大于128的
rna_sequence = {}
for i in tqdm(rna_index): #name.shapep[0]
    data = fasta[name[i]]
    onehot = Onehot()
    encoded = onehot.encode(data)
    #decoded = onehot.decode(encoded)
    count = rna_need_index[i].shape[0]
    ind = rna_need_index[i]
    scores = np.zeros([count,128,4])
    for j in range(count):
        scores[j] = encoded[ind[j]:ind[j]+128]
    rna_sequence[i] = scores

100%|██████████| 9957/9957 [00:04<00:00, 2296.36it/s]


存储 rna sequence 为X  rna icshape为y 以及rnalength和rna name
按照rna名字来挑，先保证同一个rna的不会既出现在训练集又出现测试集，然后再shuffle
所以先按照i来挑20%的作为测试集，然后分别按照索引打乱

In [42]:
#挑20%作为测试集rna  按照rna_index挑
rna_index_sample = np.copy(rna_index)
print rna_index.shape[0]
np.random.seed(1111)
np.random.shuffle(rna_index_sample)
train_index = rna_index_sample[:int(0.8*rna_index.shape[0])]
test_index= rna_index_sample[int(0.8*rna_index.shape[0]):]

9957


In [43]:
#计算存储按照需要的 i，每个i里几个片段
count_fragments_numer = {}
for i in rna_need_index.keys():
    count_fragments_numer[i] = rna_need_index[i].shape[0]

In [44]:
def prepare_data(index):
    #rna sequence 为X  rna icshape为y 以及rnalength和rna name
    count = 0
    for i in index:
        count +=count_fragments_numer[i]
    X = np.zeros([count,128,4])
    y = np.zeros([count,130]).astype('S')
    name_ = np.array([])
    rna_length_ = np.array([])
    count = 0
    for i in tqdm(index):
        X[count:count+count_fragments_numer[i]] = rna_sequence[i]
        y[count:count+count_fragments_numer[i]] = rna_icshape_whole[i]
        count +=count_fragments_numer[i]
        name_ = np.concatenate((name_,np.array([name[i]])))
        rna_length_ = np.concatenate((rna_length_,np.array([rna_length[i]])))
    return X,y,name_,rna_length_

In [46]:
X,y,name_,rna_length_ = prepare_data(train_index)
with h5py.File('data/new/12.22_CDS_rna_track_train') as f:
    f.create_dataset('X_train',data = X)
    f.create_dataset('y_train',data = y)
    f.create_dataset('name_train',data = name_)
    f.create_dataset('length_train',data = rna_length_)

100%|██████████| 7965/7965 [00:03<00:00, 2494.74it/s]


In [47]:
X,y,name_,rna_length_ = prepare_data(test_index)
with h5py.File('data/new/12.22_CDS_rna_track_test') as f:
    f.create_dataset('X_test',data = X)
    f.create_dataset('y_test',data = y)
    f.create_dataset('name_test',data = name_)
    f.create_dataset('length_test',data = rna_length_)

100%|██████████| 1992/1992 [00:00<00:00, 2468.83it/s]


此时还未打乱，然后再分别打乱片段

In [7]:
with h5py.File('data/new/12.22_CDS_rna_track_train') as f:
    X_train = f['X_train'][:]

In [9]:
X_train.shape

(1326561, 128, 4)

In [3]:
with h5py.File('data/new/12.22_CDS_rna_track_train') as f:
    X_train = f['X_train'][:]
    y_train = f['y_train'][:]
    name_train = f['name_train'][:]
    length_train = f['length_train'][:]

In [8]:
X_train[0].shape

(128, 4)

In [5]:
y_train.shape

(1326561, 130)

In [15]:
name_train[:100]

array(['ENSMUST00000200100.1-CDS', 'ENSMUST00000199311.1-CDS',
       'ENSMUST00000173895.7-CDS', 'ENSMUST00000126794.1-CDS',
       'ENSMUST00000033763.14-CDS', 'ENSMUST00000213866.1-CDS',
       'ENSMUST00000021356.5-CDS', 'ENSMUST00000020078.13-CDS',
       'ENSMUST00000112305.9-CDS', 'ENSMUST00000103118.3-CDS',
       'ENSMUST00000100484.4-CDS', 'ENSMUST00000051867.6-CDS',
       'ENSMUST00000073691.3-CDS', 'ENSMUST00000089680.9-CDS',
       'ENSMUST00000038388.5-CDS', 'ENSMUST00000155781.1-CDS',
       'ENSMUST00000135772.1-CDS', 'ENSMUST00000151708.2-CDS',
       'ENSMUST00000043584.4-CDS', 'ENSMUST00000028583.7-CDS',
       'ENSMUST00000034983.6-CDS', 'ENSMUST00000034000.14-CDS',
       'ENSMUST00000059498.11-CDS', 'ENSMUST00000029879.14-CDS',
       'ENSMUST00000022960.2-CDS', 'ENSMUST00000196528.4-CDS',
       'ENSMUST00000094378.9-CDS', 'ENSMUST00000143728.1-CDS',
       'ENSMUST00000118366.7-CDS', 'ENSMUST00000010899.13-CDS',
       'ENSMUST00000167149.7-CDS', 'ENSMUST000000

In [13]:
length_train

array([ 555.,  365.,  135., ...,  537.,  360.,  768.])

In [ ]:
#计算acc阶段，再次核查之前写的对不对

#在生成y_test 的时候要标清楚每条的rna来源以及其起始为rna的第几个position

#剩下的都是有值的点，已经知道该片段的rna和起始位点了，下面用rna一条一条计算。y_pred和y_true 都对应到rna上
#最终的y_true不是CDS里读出来的标注，因为好多位置被转成了nan
#同步统计y_true 和y_pred  每条片段的rna对应回去，np.ndarray 一个rna长度的向量
#预计新的y_true 每行[rna,origin,128*ichshape]  name指rna name

#计算y_true
def rna_icshape_true():
    position_need = {}
    rna_icshape = {}
    #每条rna每个位置出现了几次
    rna_position = {}
    rna_ic_correct = {}
    for i in range(name.shape[0]):
        #每条rna
        rna_icshape[i] = np.zeros([length[i]])
        #每个片段看看是否匹配
        rna_position[i] = np.array([])
        for j in range(y_true.shape[0]):
            if y_true[j][0] = name[i]:
                origin = y_true[j][1]
                #找不是nan的部分,要累加
                index = np.where(np.isnan(y_true[2:])=0)[0]
                rna_icshape[i][origin:origin+128][index] += y_true[j][2:][index]
                #统计赋值的位置
                position = np.arange(origin:origin+128)[index]
                rna_position[i] = np.concatenate((rna_position[i],position))
        #统计第i条rna每个位置分别被加了几次
        position_count = np.zeros([length[i]])
        unique, counts = numpy.unique(rna_position[i], return_counts=True)
        position_count[unique] = counts
        #找到position count不是0的位置
        index_ = np.where(position_count)[0]
        rna_ic_correct[i] = rna_icshape[i][index_].astype('float') / position_count[index_]
        rna_ic_correct[i][np.where(rna_ic_correct>=0.5)] =1
        rna_ic_correct[i][np.where(rna_ic_correct<0.5)] =0
        position_need[i] = index_
    return rna_ic_correct,position_need

#计算y_test
def rna_icshape_pred():
    rna_icshape = {}
    #每条rna每个位置出现了几次
    rna_position = {}
    rna_ic_correct = {}
    for i in range(name.shape[0]):
        #每条rna
        rna_icshape[i] = np.zeros([length[i]])
        #每个片段看看是否匹配
        rna_position[i] = np.array([])
        for j in range(y_true.shape[0]):
            if y_true[j][0] = name[i]:
                origin = y_true[j][1]
                #找不是nan的部分,要累加
                index = np.where(np.isnan(y_true[2:])=0)[0]
                rna_icshape[i][origin:origin+128][index] += y_pred[j][index]
                #统计赋值的位置
                position = np.arange(origin:origin+128)[index]
                rna_position[i] = np.concatenate((rna_position[i],position))
        #统计第i条rna每个位置分别被加了几次
        position_count = np.zeros([length[i]])
        unique, counts = numpy.unique(rna_position[i], return_counts=True)
        position_count[unique] = counts
        #找到position count不是0的位置
        index_ = np.where(position_count)[0]
        rna_ic_correct[i] = rna_icshape[i][index_].astype('float') / position_count[index_]
        rna_ic_correct[i][np.where(rna_ic_correct>=0.5)] =1
        rna_ic_correct[i][np.where(rna_ic_correct<0.5)] =0
    return rna_ic_correct

In [ ]:
#看下 返回的样子  需要返回一个真正被考虑的位置的index，要不然nan和0都是0，acc肯定大！ 
#读取 true和pred  以及每条rna需要被考虑的位点

In [ ]:
#分类计算acc  对应出来染色体名字和所属类别
category = []
acc = {}
for j in range(len(category)):
    acc[category[j]] = {}
    for i in range(name.shape[0]):
        if name[i] = category[j]:
            count = position_need[i].shape[0]    
            acc[category[j]][name[i]] = (count - np.sum(rna_ic_correct_true - rna_ic_correct_pred))/count    